# Getting Results (Pagination)


The CDA provides a custom python tool for searching CDA data. [`Q`](usage/#q) (short for Query) offers several ways to search and filter data, and several input modes:

---
- **<a href="../../QuickStart/usage/#q">Q.()</a>** builds a query that can be used by `run()` or `count()`
- **<a href="../../QuickStart/usage/#qrun">Q.run()</a>** returns data for the specified search 
- **<a href="../../QuickStart/usage/#qcount">Q.count()</a>** returns summary information (counts) data that fit the specified search
- **<a href="../../QuickStart/usage/#columns">columns()</a>** returns entity field names
- **<a href="../../QuickStart/usage/#unique_terms">unique_terms()</a>** returns entity field contents

---
                                                                    
Before we do any work, we need to import several functions from cdapython:
- `Q` and `query` which power the search
- `columns` which lets us view entity field names
- `unique_terms` which lets view entity field contents

We're also importing functions from several other packages to make viewing and manipulating tables easier. The `opt.` settings are pre-configuring how itables should display our tables, with scrolling and paging enabled.
Finally, we're telling cdapython to report it's version so we can be sure we're using the one we mean to:

In [1]:
from cdapython import Q, columns, unique_terms, query
import numpy as np
import pandas as pd
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=True)
import itables.options as opt
opt.maxBytes=0
opt.scrollX="200px"
opt.scrollCollapse=True
opt.paging=True
opt.maxColumns=0
print(Q.get_version())

<IPython.core.display.Javascript object>

2022.10.24

The CDA indexes tens of thousands of subjects, researchsubjects, speciments, and their diagnosis data. CDA also indexes more than 45 million files. To keep search results from being overwhelming, CDA limits search results to the first 100 records by default:

In [3]:
myquery = Q('primary_diagnosis_site = "brain"')
brainresults = myquery.subject.run()
brainresults

Getting results from database

Total execution time: 0
                        min 3.656 sec 3656 ms


            
            Offset: 0
            Count: 100
            Total Row Count: 3015
            More pages: True
            


---

- **Offset:** This is how many rows of information we've told the query to skip in the data, here we didn't tell it to skip anything, so the offset is zero
- **Count:** This is how many rows the current page of our results table has. To keep searches fast, we default to pages with 100 rows.
- **Total Row Count:** This is how many rows are in the full results table
- **More pages:** This is always a True or False. False means that our current page has all the available results. True means that we will see only the first 100 results in this table, and will need to page through for more.

---

This preview behaviour is great for search, but not when you are ready to download the information. The pagination feature is how you retrieve all the results for your final query and not just the first 100. 

You can have your full results output to a dataframe or a list.

## Results to a dataframe

The easiest way to get results to a dataframe is to use the `auto_paginator()`, which directly turns your results into a full dataframe:


In [4]:
paginated_brain_results = brainresults.auto_paginator(to_df=True)

Output()

In [5]:
paginated_brain_results # view the dataframe

subject_id                                 subject_identifier  \
0          TCGA-14-1454  [{'system': 'GDC', 'value': 'TCGA-14-1454'}, {...   
1          TCGA-16-1060  [{'system': 'GDC', 'value': 'TCGA-16-1060'}, {...   
2          TCGA-DU-5870  [{'system': 'GDC', 'value': 'TCGA-DU-5870'}, {...   
3          TCGA-DU-6397  [{'system': 'GDC', 'value': 'TCGA-DU-6397'}, {...   
4          TCGA-DU-7009  [{'system': 'GDC', 'value': 'TCGA-DU-7009'}, {...   
..                  ...                                                ...   
95              C881049            [{'system': 'PDC', 'value': 'C881049'}]   
96  GENIE-GRCC-159a16b7  [{'system': 'GDC', 'value': 'GENIE-GRCC-159a16...   
97  GENIE-MSK-P-0005515  [{'system': 'GDC', 'value': 'GENIE-MSK-P-00055...   
98  QIN-BRAIN-DSC-01-33  [{'system': 'IDC', 'value': 'QIN-BRAIN-DSC-01-...   
99        QIN-GBM-TR-13      [{'system': 'IDC', 'value': 'QIN-GBM-TR-13'}]   

         species     sex                    race               ethnicity  \
0   homo sapiens  female                   white            not reported   
1   homo sapiens  female                   white  not hispanic or latino   
2   homo sapiens  female                   white  not hispanic or latino   
3   homo sapiens    male                   white  not hispanic or latino   
4   homo sapiens  female                   white  not hispanic or latino   
..           ...     ...                     ...                     ...   
95  homo sapiens    male                   white  not hispanic or latino   
96  homo sapiens  female  not allowed to collect  not allowed to collect   
97  homo sapiens    male                   white  not hispanic or latino   
98  homo sapiens    None                    None                    None   
99  homo sapiens    None                    None                    None   

    days_to_birth                         subject_associated_project  \
0             NaN                               [TCGA-GBM, tcga_gbm]   
1             NaN                               [TCGA-GBM, tcga_gbm]   
2             NaN                               [tcga_lgg, TCGA-LGG]   
3             NaN                               [tcga_lgg, TCGA-LGG]   
4             NaN                               [tcga_lgg, TCGA-LGG]   
..            ...                                                ...   
95            NaN  [Proteogenomic Analysis of Pediatric Brain Can...   
96            NaN                                       [GENIE-GRCC]   
97            NaN                                        [GENIE-MSK]   
98            NaN                                [qin_brain_dsc_mri]   
99            NaN                       [qin_gbm_treatment_response]   

    vital_status days_to_death cause_of_death  
0           Dead          None           None  
1           Dead          None           None  
2          Alive          None           None  
3           Dead          None           None  
4           Dead          None           None  
..           ...           ...            ...  
95         Alive           NaN   Not Reported  
96  Not Reported           NaN           None  
97  Not Reported           NaN           None  
98          None           NaN           None  
99          None           NaN           None  

[3015 rows x 11 columns]

If you'd like more control over your output, you can use the `paginator`. It will also get all the data, but requires you to write the looping code.

In the simplest case, you create an empty dataframe for the data to land in, then use the paginator in a loop to get all the results:

In [6]:
mydf = pd.DataFrame()
for i in myquery.subject.run().paginator(to_df=True):
    mydf = pd.concat([mydf, i])

Getting results from database

Total execution time: 0
                        min 3.652 sec 3652 ms

Output()

In [7]:
mydf  # view the dataframe

subject_id                               subject_identifier  \
0           C177366          [{'system': 'PDC', 'value': 'C177366'}]   
1         C3N-03003        [{'system': 'IDC', 'value': 'C3N-03003'}]   
2            C48216           [{'system': 'PDC', 'value': 'C48216'}]   
3           C794457          [{'system': 'PDC', 'value': 'C794457'}]   
4            C82041           [{'system': 'PDC', 'value': 'C82041'}]   
..              ...                                              ...   
10  UPENN-GBM-00230  [{'system': 'IDC', 'value': 'UPENN-GBM-00230'}]   
11  UPENN-GBM-00307  [{'system': 'IDC', 'value': 'UPENN-GBM-00307'}]   
12  UPENN-GBM-00419  [{'system': 'IDC', 'value': 'UPENN-GBM-00419'}]   
13  UPENN-GBM-00562  [{'system': 'IDC', 'value': 'UPENN-GBM-00562'}]   
14              W16              [{'system': 'IDC', 'value': 'W16'}]   

         species     sex                              race  \
0   homo sapiens    male  american indian or alaska native   
1   homo sapiens    None                              None   
2   homo sapiens  female                             white   
3   homo sapiens    male                             white   
4   homo sapiens    male                      not reported   
..           ...     ...                               ...   
10  homo sapiens    None                              None   
11  homo sapiens    None                              None   
12  homo sapiens    None                              None   
13  homo sapiens    None                              None   
14  homo sapiens    None                              None   

                 ethnicity  days_to_birth  \
0   not hispanic or latino            NaN   
1                     None            NaN   
2   not hispanic or latino            NaN   
3   not hispanic or latino            NaN   
4   not hispanic or latino            NaN   
..                     ...            ...   
10                    None            NaN   
11                    None            NaN   
12                    None            NaN   
13                    None            NaN   
14                    None            NaN   

                           subject_associated_project vital_status  \
0   [Proteogenomic Analysis of Pediatric Brain Can...         Dead   
1                                         [cptac_gbm]         None   
2   [Proteogenomic Analysis of Pediatric Brain Can...        Alive   
3   [Proteogenomic Analysis of Pediatric Brain Can...        Alive   
4   [Proteogenomic Analysis of Pediatric Brain Can...        Alive   
..                                                ...          ...   
10                                        [upenn_gbm]         None   
11                                        [upenn_gbm]         None   
12                                        [upenn_gbm]         None   
13                                        [upenn_gbm]         None   
14                                           [ivygap]         None   

   days_to_death cause_of_death  
0            NaN   Not Reported  
1            NaN           None  
2            NaN   Not Reported  
3            NaN   Not Reported  
4            NaN   Not Reported  
..           ...            ...  
10          None           None  
11          None           None  
12          None           None  
13          None           None  
14          None           None  

[3015 rows x 11 columns]

## Results to a list

If you want results in a list, you'll need to use the paginator. Saving to a list works similarly to the dataframe call. The differences are:

- initiate a list not a DataFrame
- change `to_list=True`
- change the concat index to `extend()`

In [8]:
mylist = []
for i in myquery.subject.run().paginator(to_list=True):
    mylist.extend(i)

Getting results from database

Total execution time: 0
                        min 3.654 sec 3654 ms

Output()

This gives back the correct number of results:

In [9]:
len(mylist)

3015

And we can preview the first result to see that it has the same values:

In [10]:
mylist[0:1]

[{'subject_id': 'C177366',
  'subject_identifier': [{'system': 'PDC', 'value': 'C177366'}],
  'species': 'homo sapiens',
  'sex': 'male',
  'race': 'american indian or alaska native',
  'ethnicity': 'not hispanic or latino',
  'days_to_birth': None,
  'subject_associated_project': ['Proteogenomic Analysis of Pediatric Brain Cancer Tumors Pilot Study'],
  'vital_status': 'Dead',
  'days_to_death': None,
  'cause_of_death': 'Not Reported'}]